# Spin-Spin Hamiltonian Gate Fitting, with Decoherence

In [1]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from timedependent_channels import *
from utils import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=5, suppress=True)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


# Spin-Spin Hamiltonian, Noise Fitting

## Target Channel

In [2]:
tf.random.set_seed(42)
np.random.seed(42)

jump_operator = JumpOperator(4, trainable=False)
H_target = SpinSpin(degree=2)

lindblad_target = LindbladGenerator(hamiltonian = H_target, 
                                   jump_operator = jump_operator,
                                   gamma = 0,
                                 )

channel_target = MagnusPropagator(liouvillian=lindblad_target, grid_size=200, T = 1)

## No Decoherence Optimization

In [3]:
tf.random.set_seed(43)
np.random.seed(43)

jump_operator = JumpOperator(4, trainable=False)
H_model = SpinSpin(degree=4)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 0,
                                 )
channel_model1 = MagnusPropagator(liouvillian=lindblad_model, grid_size=100, T = 1)

In [4]:
model1 = ModelQuantumMap(channel = channel_model1,
                        loss_function = channel_mse_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss, sample_freq=1, N=0),
                        )

In [5]:
model1.train(inputs=None,
             targets=[channel_target],
             num_iter=500,
             N=0,
             verbose=False,)

-1.000000073614705 None


## Decoherence Optimization

In [6]:
tf.random.set_seed(43)
np.random.seed(43)

jump_operator = JumpOperator(4, trainable=True)
H_model = SpinSpin(degree=4)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 1,
                                 )
channel_model2 = MagnusPropagator(liouvillian=lindblad_model, grid_size=100, T = 1)

In [7]:
model2 = ModelQuantumMap(channel = channel_model2,
                        loss_function = channel_mse_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss, sample_freq=1, N=0),
                        )

In [8]:
model2.train(inputs=None,
            targets=[channel_target],
            num_iter=500,
            N=0,
            verbose=False,)

  0%|          | 0/500 [00:00<?, ?it/s]

tf.Tensor(0.9217717775651921, shape=(), dtype=float64)


In [21]:
channel_model1.liouvillian.gamma=1
print(channel_fidelity(channel_target, channel_model1))
print(channel_fidelity(channel_target, channel_model2))

tf.Tensor(0.8309002085044421, shape=(), dtype=float64)
tf.Tensor(0.8466703099769655, shape=(), dtype=float64)


## Noise Fitting with Jump Operators with Non-Zero Trace

In [13]:
tf.random.set_seed(43)
np.random.seed(43)

jump_operator = JumpOperator(4, trainable=True, trace_less = False)
H_model = SpinSpin(degree=4)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 1,
                                 )
channel_model3 = MagnusPropagator(liouvillian=lindblad_model, grid_size=100, T = 1)

In [14]:
model3 = ModelQuantumMap(channel = channel_model3,
                        loss_function = channel_mse_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss, sample_freq=1, N=0),
                        )

In [15]:
model3.train(inputs=None,
            targets=[channel_target],
            num_iter=500,
            N=0,
            verbose=False,)

  0%|          | 0/500 [00:00<?, ?it/s]

tf.Tensor(0.11924420167019795, shape=(), dtype=float64)


In [18]:
print(channel_fidelity(channel_target, channel_model3))

tf.Tensor(0.9137084761060119, shape=(), dtype=float64)


In [27]:
print(tf.linalg.trace(channel_model3.liouvillian.JumpOperator([0])[0]))
print(tf.linalg.trace(channel_model2.liouvillian.JumpOperator([0])[0]))

0.4173989595701445
tf.Tensor((-1.3877787807814457e-17-6.938893903907228e-18j), shape=(), dtype=complex128)
